<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 6. Примеры</h1></div>

## Стохастическое моделирование
### Пакет SimJulia
### Пример с клиентским обслуживанием в банке

In [23]:
#
# bank_teller.jl
#

using SimJulia
using Distributions

function customer(env::Environment, name::UTF8String, counter::Resource, time_in_bank::Float64)
  arrive = now(env)
    
  println("$arrive $name: прибыл")
    
  req = Request(counter)
  patience = rand(Uniform(MIN_PATIENCE, MAX_PATIENCE))
  result = yield(req | Timeout(env, patience))
    
  wait = now(env) - arrive
  push!(wait_times, wait);
    
  if in(req, keys(result))
    println("$(now(env)) $name: прождал $wait") 
    yield(Timeout(env, rand(Exponential(time_in_bank))))
    println("$(now(env)) $name: обслужен")    
    yield(Release(counter))
    #push!(in_bank_times, now(env) - arrive);
  else
    println("$(now(env)) $name: ОТКАЗАЛСЯ после $wait")
    push!(renege_qty, 1);       
    SimJulia.cancel(req)
  end
end

function source(env::Environment, number::Int, interval::Float64, counter::Resource)
  d = Exponential(interval)
  for i in 1:number
    Process(env, customer, "Клиент №$i", counter, 12.0)
    yield(Timeout(env, rand(d)))
  end
end

function randomize()  # lcg_rand
  seed = ccall( (:GetTickCount, "kernel32"), stdcall, Int32, () );
  ((seed * 214013 + 2531011) >> 16) & 0x7fff
end

# Настроить и запустить имитационный прогон
println("Отказы от услуги в банке\n")

# Установить исходные данные имитации 

const MAX_CUSTOMERS     = 20    # Общее число клиентов
const MAX_TIME          = 300.0 # Общее время работы банка
const MEAN_ARRIVAL_TIME = 5.0   # Генерировать новых клиентов с прим. промежутком x (интервалы)
const MIN_PATIENCE      = 1.0   # Минимальное терпение клиента
const MAX_PATIENCE      = 8.0   # Максимальное терпение клиента (срд.время обслуживания)
const MAX_TELLERS       = 2     # Число операционистов

# Использовать ранее определенную функцию randomize() либо 
# выбрать значение, чтобы сделать прогон воспроизводимым
#const RANDOM_SEED = 150  
const RANDOM_SEED = randomize()
srand(RANDOM_SEED)

global wait_times = Float64[];  # Времена ожидания
global renege_qty = Int32[];    # Количество отказов

env = Environment()

# Запустить процессы и выполнить
counters = Resource(env, MAX_TELLERS)
Process(env, source, MAX_CUSTOMERS, MEAN_ARRIVAL_TIME, counters)
run(env, MAX_TIME)

@printf "\nПри наличии %s операциониста(ов):\n" MAX_TELLERS
@printf "Среднее время ожидания = %5.3f\n" mean(wait_times)
@printf "Количество отказов = %5d" sum(renege_qty)

Отказы от услуги в банке

0.0 Клиент №1: прибыл
0.0 Клиент №1: прождал 0.0
0.5675342114407739 Клиент №1: обслужен
2.5816273243153756 Клиент №2: прибыл
2.5816273243153756 Клиент №2: прождал 0.0
5.28159974894999 Клиент №2: обслужен
15.494063559677903 Клиент №3: прибыл
15.494063559677903 Клиент №3: прождал 0.0
16.874762086683333 Клиент №4: прибыл
16.874762086683333 Клиент №4: прождал 0.0
20.309905004860234 Клиент №4: обслужен
21.99409426964462 Клиент №5: прибыл
21.99409426964462 Клиент №5: прождал 0.0
22.154520340374468 Клиент №6: прибыл
24.850694427298027 Клиент №5: обслужен
24.850694427298027 Клиент №6: прождал 2.696174086923559
28.52902036203353 Клиент №6: обслужен
31.367609061486462 Клиент №3: обслужен
34.89793093679844 Клиент №7: прибыл
34.89793093679844 Клиент №7: прождал 0.0
45.22374220866823 Клиент №7: обслужен
50.15466688217829 Клиент №8: прибыл
50.15466688217829 Клиент №8: прождал 0.0
51.440947124916946 Клиент №8: обслужен
54.9281466152149 Клиент №9: прибыл
54.9281466152149 Клие